In [1]:
import os 
import sys
import pickle
path_lobster = os.path.abspath('./')
path_data = path_lobster+'/data'
sys.path.append(path_lobster+'/src')

In [2]:
import numpy as np
import pandas as pd
import prepare_from_lobster as from_lobster

In [3]:
symbol='MSFT'
date='2012-06-21'
initial_time=float(9.0*60*60)
final_time=float(16*60*60)
time_window=str('{}-{}'.format(int(initial_time),int(final_time)))
first_read_fromLOBSTER=True
dump_after_reading=False
add_level_to_messagefile=True

In [4]:
print('symbol={}, date={}, time_window={}\n'.format(symbol,date,time_window))
if (first_read_fromLOBSTER):
    LOB,messagefile=from_lobster.read_from_LOBSTER(symbol,date,
                                      dump_after_reading=dump_after_reading,
                                      add_level_to_messagefile=add_level_to_messagefile)
else:
    LOB,messagefile=from_lobster.load_from_pickleFiles(symbol,date)

symbol=MSFT, date=2012-06-21, time_window=32400-57600

Given shape of orderbook: 668765x40
Given shape of messagefile: 668765x6
read_from_LOBSTER: time window:
  initial_time=34200.015625
  final_time=57599.90625
I am adding 'level' to messagefile
declare_level: check = True


In [5]:
LOB,messagefile=from_lobster.select_subset(LOB,messagefile,
                              initial_time=initial_time,
                              final_time=final_time)

In [6]:
n_levels = 2 #Number of levels to take into account in the volumebook
aggregate_time_stamp=True
eventTypes_to_aggregate=[1,2,3,4,5]
eventTypes_to_drop_with_nonunique_time=[]
eventTypes_to_drop_after_aggregation=[3]
only_4_events=False
separate_directions=True
separate_13_events=False
separate_31_events=False
separate_41_events=True
separate_34_events=True
separate_43_events=True
equiparate_45_events_with_same_time_stamp=True
drop_all_type3_events_with_nonunique_time=False
drop_5_events_with_same_time_stamp_as_4=True
drop_5_events_after_aggregation=True
tolerance_when_dropping=1.0e-8
clear_same_time_stamp=True
num_iter=4

In [7]:
print('\n\nDATA CLEANING\n')

man_mf=from_lobster.ManipulateMessageFile(
     LOB,messagefile,
     symbol=symbol,
     date=date,
     n_levels = n_levels,
     aggregate_time_stamp=aggregate_time_stamp,
     eventTypes_to_aggregate=eventTypes_to_aggregate,  
     only_4_events=only_4_events,
     separate_directions=separate_directions,
     separate_13_events=separate_13_events,
     separate_31_events=separate_31_events,
     separate_41_events=separate_41_events,
     separate_34_events=separate_34_events,
     separate_43_events=separate_43_events,
     equiparate_45_events_with_same_time_stamp=
     equiparate_45_events_with_same_time_stamp,
     eventTypes_to_drop_with_nonunique_time=eventTypes_to_drop_with_nonunique_time,
     eventTypes_to_drop_after_aggregation=eventTypes_to_drop_after_aggregation,
     drop_all_type3_events_with_nonunique_time=drop_all_type3_events_with_nonunique_time,  
     drop_5_events_with_same_time_stamp_as_4=drop_5_events_with_same_time_stamp_as_4,
     drop_5_events_after_aggregation=drop_5_events_after_aggregation,
     tolerance_when_dropping=tolerance_when_dropping,
     clear_same_time_stamp=clear_same_time_stamp,
     num_iter=num_iter
)


man_ob=from_lobster.ManipulateOrderBook(
    man_mf.aggregated_LOB,symbol=symbol,date=date,
    ticksize=man_mf.ticksize,n_levels=man_mf.n_levels,volume_imbalance_upto_level=2)


data=from_lobster.DataToStore(man_ob,man_mf,time_origin=initial_time)



DATA CLEANING

Initial length of the message file: 668765
ManipulateMessageFile.aggregate_time_stamp: contraction. only_4_events=False
  From 83922 lines with same time stamp, same event type and same direction, -->  to 36306 lines: 47616 lines contracted
  Initial length=668765, contracted length=621149
  Number of remaining events with same time stamp=11139
I am dropping events with non-unique time stamp and event type=3, while keeping the_last of the events in every batch
Numbers of events dropped: 527
I am dropping events with non-unique time stamp and event type=5, while keeping the_last of the events in every batch
Numbers of events dropped: 940
Total number of events dropped: 1467
prepare_from_lobster.separate_41_events: number of occurrencies = 154 
   After separation, the average time step is 0.09416134604599855, the smallest time step is 1.6899866750463843e-07.
prepare_from_lobster.separate_34_events: number of occurrencies = 439 
   After separation, the average time step

In [8]:
data.messagefile

,direction,event_type,level,original_idx,price,size,time,delta_t,same_time_stamp
0,-1,1,6,1,310500,200,34200.013994,34200.013994,False
1,-1,1,6,2,310400,100,34200.015248,0.001254,False
2,-1,1,7,3,310500,100,34200.015442,0.000194,False
3,-1,1,8,4,310600,100,34200.015789,0.000347,False
4,-1,1,9,5,310700,100,34200.016300,0.000510,False
...,...,...,...,...,...,...,...,...,...
612011,-1,3,3,668759,301600,500,57599.782541,0.141719,False
612012,-1,3,1,668760,301400,9249,57599.861466,0.078925,False
612013,-1,3,1,668761,301400,783,57599.861567,0.000101,False
612014,-1,3,2,668762,301500,800,57599.873177,0.011611,False


In [9]:
data.LOB

,ask_price_1,ask_volume_1,bid_price_1,bid_volume_1,ask_price_2,ask_volume_2,bid_price_2,bid_volume_2,ask_price_3,ask_volume_3,...,ask_price_9,ask_volume_9,bid_price_9,bid_volume_9,ask_price_10,ask_volume_10,bid_price_10,bid_volume_10,original_idx,time
0,309900,3788,309500,300,310500,200,309300,3986,310600,100,...,311300,100,308500,400,311400,100,308400,1600,1,34200.013994
1,309900,3788,309500,300,310400,100,309300,3986,310500,200,...,311100,4500,308500,400,311300,100,308400,1600,2,34200.015248
2,309900,3788,309500,300,310400,100,309300,3986,310500,300,...,311100,4500,308500,400,311300,100,308400,1600,3,34200.015442
3,309900,3788,309500,300,310400,100,309300,3986,310500,300,...,311100,4500,308500,400,311300,100,308400,1600,4,34200.015789
4,309900,3788,309500,300,310400,100,309300,3986,310500,300,...,311100,4500,308500,400,311300,100,308400,1600,5,34200.016300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612011,301400,26632,301300,50426,301500,81463,301200,83306,301600,65399,...,302200,36532,300500,23446,302300,31600,300400,36675,668759,57599.782541
612012,301400,17383,301300,50426,301500,81463,301200,83306,301600,65399,...,302200,36532,300500,23446,302300,31600,300400,36675,668760,57599.861466
612013,301400,16600,301300,50426,301500,81463,301200,83306,301600,65399,...,302200,36532,300500,23446,302300,31600,300400,36675,668761,57599.861567
612014,301400,16600,301300,50426,301500,80663,301200,83306,301600,65399,...,302200,36532,300500,23446,302300,31600,300400,36675,668762,57599.873177


In [10]:
data.volumebook

,original_idx,time,ask_volume_1,bid_volume_1,ask_volume_2,bid_volume_2,tot_vol
0,1,34200.013994,0.457820,0.036258,0.024172,0.481750,8274
1,2,34200.015248,0.463421,0.036702,0.012234,0.487644,8174
2,3,34200.015442,0.463421,0.036702,0.012234,0.487644,8174
3,4,34200.015789,0.463421,0.036702,0.012234,0.487644,8174
4,5,34200.016300,0.463421,0.036702,0.012234,0.487644,8174
...,...,...,...,...,...,...,...
612011,668759,57599.782541,0.110128,0.208521,0.336865,0.344486,241827
612012,668760,57599.861466,0.074741,0.216813,0.350261,0.358185,232578
612013,668761,57599.861567,0.071615,0.217546,0.351444,0.359395,231795
612014,668762,57599.873177,0.071863,0.218299,0.349198,0.360640,230995
